# Home Assignment -- 2

Please, write your solutions within the designated blocks:
```python
...
### BEGIN Solution

# >>> your solution here <<<

### END Solution
...
```

Import visualization and data processing libraries

In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

<br/> <!--Intentionally left blank-->

# Part 1 (33 pt.): Convex Programs and Support Vector Models

In this **first two tasks** of this part of the homework you will implement
your own Support Vectors Regression and then test on a simple real dataset.

In the last task you are asked to solve a simple small NLP problem using
support vector classfiers from [scikit-learn](http://scikit-learn.org/stable/modules/svm.html#classification).
If you successfully complete the task, it will give you a ready-to-use and
straighforward baseline for any NLP classification task of moderate size.

### Support Vector Regression

$$\frac{1}{2}\|w\|^2 + C\sum_{i=1}^l(\xi_i + \xi_i') \to \min_{w, b, \xi_i, \xi_i'}\\
(w^Tx_i + b) - y_i \leq \epsilon - \xi_i\\
y_i - (w^Tx_i + b) \leq \epsilon - \xi_i'\\
\xi_i \geq 0\\
\xi_i' \geq 0$$

This problem can be written as empirical risk minimization problem with regularization

#### Equivalent
$$ \frac{1}{2}\|w\|^2 + C\sum_{i=1}^l h(\hat{y}_i, y_i)$$



Where
$$ h(\hat{y}_i, y_i) = \max(0, |\hat{y} - y| - \epsilon ) $$
is the $\epsilon$-insensitive loss, and
$$ \hat{y}_i = (w, x_i) + b $$
is the prediction.

For our experiments we will use "Cars" dataset, which contains information about braking distances for several cars from the 1920s.

In [ ]:
cars_info = pd.read_csv('data/cars.csv', index_col=0, dtype=np.float)

X = cars_info.speed.values.reshape(-1, 1)

Normalization leads to better convergence

In [ ]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler()

X_train = X_scaler.fit_transform(X)
y_train = cars_info.dist

In [ ]:
ax = plt.gca()
ax.plot(X, y_train, 'o')
ax.set_xlabel('Speed')
ax.set_ylabel('Distance')

<br/> <!--Intentionally left blank-->

## Task 1 (11 pt.)

Please implement a function that calculates the current loss value and its gradient
at the specified values of weights $w$, bias $b$, selected $C$ and $\epsilon$.

In [ ]:
def calculate_eps_insensetive_loss(X, y, w, b, C=1.0, eps=0.1):
    '''Calculate hinge loss function
    :param: X -- numpy array of shape l x n
    :param: y -- numpy array of shape l with values 1 and -1
    :param: w -- numpy array of shape n
    :param: b -- np.float64
    :param: C -- np.float64 
    '''
    # Calculate Loss
    loss = 0
    ### BEGIN Solution

    ### END Solution

    # Calculate Gradient for hinge loss    
    w_grad = np.zeros_like(w)
    b_grad = 0
    ### BEGIN Solution

    ### END Solution

    return loss, w_grad, b_grad

Following code finds optimal values for your loss function with simple constant step gradient descend.

In [ ]:
# Model's metaparameters
eps = 0.1
C = 100

# Gradient descent parameters
step = 1e-4
tolerance = 1e-3

# Optimization logging
converged = False
index = 0
training_loss_history = []
training_loss_iteration = []

# Parameters initialization
w = np.zeros(1)
b = 0
loss, w_grad, b_grad = calculate_eps_insensetive_loss(X_train, y_train, w, b, C, eps)

while not converged:
    # Update parameters
    new_w = w - step * w_grad
    new_b = b - step * b_grad
    # Calculate new loss and gradient
    new_loss, w_grad, b_grad = calculate_eps_insensetive_loss(X_train, 
                                                    y_train, 
                                                    new_w, new_b, C, eps)
    # Check convergence
    if np.abs(loss - new_loss) < tolerance:
        converged = True
    
    b, w, loss = new_b, new_w, new_loss
    
    # Update history
    index += 1
    if index % 10 == 0:
        training_loss_history.append(new_loss)
        training_loss_iteration.append(index)

In [ ]:
axis = plt.gca()
axis.plot(training_loss_iteration, training_loss_history)
axis.set_xlabel('Iterations')
axis.set_ylabel('Loss value')

We are going calculate predictions for all speeds in range from $0$ to $30$.

In [ ]:
x_grid = np.arange(0., 30., 1).reshape(-1, 1)
x_test = X_scaler.transform(x_grid)

In [ ]:
predictions = x_test @ w + b

In [ ]:
ax = plt.gca()
ax.plot(x_grid, predictions, label='Predictions')
ax.plot(X, y_train, 'o')
ax.set_xlabel('Speed')
ax.set_ylabel('Distance')
ax.legend(loc='best')

<br/> <!--Intentionally left blank-->

## Task 2 (11 pt.)

In this task the goal is to implement the SVR algorithm based on its dual problem version.

### SVR Dual Problem

\begin{equation}
    \begin{aligned}
        & \underset{\alpha', \alpha}{\text{minimize}}
          & & \frac{1}{2}(\alpha' - \alpha)^T  K (\alpha' - \alpha)
              - (\alpha' - \alpha)^T Y
              + \epsilon (\alpha' + \alpha)^T \mathbf{1}
              \,, \\
        & \text{subject to}
          & & (\alpha' - \alpha)^T \mathbf{1} = 0
              \,, \\
        & & & \alpha', \alpha \in [0, C]
              \,.
    \end{aligned}
\end{equation}

### Quadtratic Optimization Problem

\begin{equation}
    \begin{aligned}
        & \underset{x}{\text{minimize}}
          & & \tfrac12 x^T P x + q^T x
              \,, \\
        & \text{subject to}
          & & G x \leq h
              \,, \\
        & & & Ax = b
              \,.
    \end{aligned}
\end{equation}

Please notice that instead of just $\alpha$, like in the classification case, we have
two vectors of coefficients: $\alpha$ and $\alpha'$. As an input space consider using
stacked column vectors
\begin{equation}
    \begin{pmatrix}
        \alpha \\
        \alpha'
    \end{pmatrix}\,.
\end{equation}

**Hint**: You may find function `np.block` very usefull in this task

Now we are going to use **cvxopt** library, which has solvers for different kind
of convex optimization problems. Please notice that installation in Windows can
be a little tricky and painful.

http://cvxopt.org/

For more information about different types of optimization problem you can check
this presentations. This material could be very helpful, however is not necessary
for understatnding this course in Machine Learning.

http://ee364a.stanford.edu/lectures/problems.pdf

### Reformulate the Dual Problem

In [ ]:
from cvxopt import matrix


def formulate_problem(X, y, kernel, C, epsilon):
    """Formulate quadratic optimization
    problem by defining matrices
    :param: X np.array of size n_elem * n_dim with reaining values
    :param: y np.array of size n_elem with labels 1 and -1
    :kernel: kernel function wich accepts two matrices of objects
    """
    quadratic_problem = {}

    # REMEMBER: all values should be wrapped in `cvxopt.matrix`
    # datatype (you can just use matrix(a) where a is `np.array`).
    # Please be careful cvxopt accepts only FLOAT not INTEGER data

    ### BEGIN Solution
    quadratic_problem['P'] = None
    quadratic_problem['q'] = None

    quadratic_problem['G'] = None
    quadratic_problem['h'] = None

    quadratic_problem['A'] = None
    quadratic_problem['b'] = None

    ### END Solution

    return quadratic_problem

### Check the Results

The module **metrics.pairwise** in **sklearn** has a very fast and well documented
implementation of kernel matrix calculator. We fix the width of the RBF kernel at
$1.0$ for simplicity.

In [ ]:
from sklearn.metrics.pairwise import rbf_kernel
from functools import partial

# the width of the RBF kernel is 1.0
kernel = partial(rbf_kernel, gamma=0.1)

**cvxopt** has powerful and versatile solvers for almost every convex
optimization problem: linear, conic, and **quadratic**. We are using
the latter.

In [ ]:
# Optimization routine
from cvxopt.solvers import qp

qudratic_problem = formulate_problem(X_train, y_train.astype(float), kernel, C, eps)

results = qp(**qudratic_problem)

Extract optimal point form the results and force **small values** of the coefficients to **zero**.

In [ ]:
optimal_stacked_alpha = np.array(results['x']).ravel()

zero_threshold = 1e-6
### BEGIN Solution
# force negligible values to be exactly zero

### END Solution

### Prediction

Having a kernel function $K$ and optimal $(\alpha, \alpha')$, implement a function
that computes the dot product $(w \cdot x_i)$:

$$(w \cdot x_{test}) = \sum_{i=1}^l(\alpha_i' - \alpha_i)K(x_i, x_{test}) \,. $$

In [ ]:
def kernel_dot_product(X_train, y_train, X_test, kernel, alpha):
    '''Calculate scalar product with vector w
    based on support vectors and dual coefficients
    :param: X_train np.array with train data
    :param: y_train np.array with train labels
    :param: X_test np.array with test data
    :parma: kernel the kernel function
    :param: alpha stacked dual coefficients
    '''
    output = np.zeros(len(X_test))

    ### BEGIN Solution

    ### END Solution

    return output

### Constant Calculation

The optimal constant $b$ can be derived from the theoretical properties of
the support vectors:
$$ b = y_i + \epsilon - (w \cdot x_i) \, \text{ if }\, 0 < \alpha_i < C \,, $$
and
$$ b = y_i - \epsilon - (w \cdot x_i) \, \text{ if }\, 0 < \alpha_i' < C \,. $$

For example, on infinitely powerful hardware with infinite precision arithmetic
you could have used any single support vector to get $b$. But it won't work IRL,
where the most you can hope for is [IEEE 754](https://en.wikipedia.org/wiki/IEEE_754)
quadruple precision [floating point numbers](https://en.wikipedia.org/wiki/Floating-point_arithmetic])
(setting aside special libraries or hardware). In the **numpy-scipy-sklearn** stack
on a typical x86-64 hardware your algorithms work in finite precision arithmetic, which
is subject to roundoff errors. The arithmetic results are pretty accurate, but still
**inexact**. Besides the numerical optimization algorithms (**cvxopt.qp**, **BFGS**,
**Your own SGD** and the like) themselves aren't exact, and produce just an **extremely
faithful** approximation of the optmial coefficients.

Your task here is to use as many support vectors as possible to get an estimate of $b$.

In [ ]:
def calculate_bias(X_train, y_train, kernel, alpha, eps=0.1):
    """Calculates the bias 
    :param: X_train np.array with train data
    :param: y_train np.array with train labels
    :parma: kernel the kernel function
    :param: alpha stacked dual coefficients'''
    """
    b = 0.0

    ### BEGIN Solution

    ### END Solution

    return b

In [ ]:
b = calculate_bias(X_train, y_train, kernel, optimal_stacked_alpha, eps)

Now we use the optimal $b$ and $\alpha, \alpha'$ to get the predictions and finally plot
a nice picture of the SVR.

In [ ]:
predictions = kernel_dot_product(X_train, y_train, x_test, kernel, optimal_stacked_alpha) + b

In [ ]:
ax = plt.gca()
ax.plot(x_grid, predictions, label='Predictions')
ax.plot(X, y_train, 'o')
ax.set_xlabel('Speed')
ax.set_ylabel('Distance')
ax.legend(loc='best')

<br/> <!--Intentionally left blank-->

## Task 3 (11 pt.)

In this task we are going to use SVM for text classification. We will work with
dataset which  contains different tweets with the related conversation context.
Some of the tweets are generated by bots, others -- written by humans.

In [ ]:
train_data = pd.read_csv('data/bot_or_not_train.csv', sep='\t', index_col=0)

test_data = pd.read_csv('data/bot_or_not_test.txt', sep='\t', index_col=0)

In [ ]:
train_data.head()

Please notice that context and response columns contains raw text, which is not
directly suitable for machine learning.

### Task 3.1 (6 pt.)

To extract features we are going to use the **bag of words** techniques. Please read
the documentation for `CountVectorizer` [in scikit](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)
and do the following:

* For both the context and the response generate a set of features using
two separate count vectorizers.

* Join this sets of features into one feature matrix (please notice that
`CountVectorizer` returns Scipy [CSR sparse matrices](https://en.wikipedia.org/wiki/Sparse_matrix#Compressed_sparse_row_(CSR,_CRS_or_Yale_format)).

* Pick the optimal $C$ for the **linear kernel** using cross valudation
`GridSearchCV` and `ROC AUC` as the target metric.

* Choose the optimal `gamma` and $C$ for the RBF kernel using cross valudation
`GridSearchCV` targeting `ROC AUC` metric.

* Take the **best models** with linear and RBF kernels, plot **their ROC curves**
on the provided heldout test data on the same figure, and print their `ROC AUC`
score. Which models work better on the test dataset?

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC

Generate feature matricies for the context and the response. Fill free to create additional
feature extraction objects if you like.

In [ ]:
y_train = train_data['human-generated']

### BEGIN Solution
x_context = None

x_respose = None

# Join the feature matrix
x_train = None

### END Solution

Generate the response and contet test sets with their **relevant** feature encodings.

In [ ]:
y_test = test_data['human-generated']

### BEGIN Solution
x_context_test = None

x_response_test = None

x_test = None
### END Solution

Select the best linear model among $C \in [\mathtt{1.e-3}, .., \mathtt{1.e3}]$.

In [ ]:
### BEGIN Solution
# pick the best C

best_linear_model = None

### END Solution

Select the best RBF kernel model among the hyperparameters
$C \in [\mathtt{1.e-3}, .., \mathtt{1.e3}]$ and $\gamma \in [\mathtt{1.e-3}, .., \mathtt{1.e3}]$.

In [ ]:
### BEGIN Solution
# pick the best C and gamma

best_kernel_model = None

### END Solution

Plot the `ROC` curves of both models and their `ROC AUC` scores.

In [ ]:
### BEGIN Solution

### END Solution

... and the best model is...?

In [ ]:
best_model = None
test_predictions = best_model.decision_function(x_test)
test_score = roc_auc_score(y_test, test_predictions)
print("Test ROC AUC = {}".format(test_score))

### Task 3.2 (5 pt.)

Now you are asked to **redo the last task (3.1)**, bit this time with more advanced
features generated by [TfidfVectorizer](https://en.wikipedia.org/wiki/Tf%E2%80%93idf).
To do this you can use `TfidfVectorizer` from sklearn.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

Generate feature matricies for context and response. Fill free to create
additional feature extraction object if necessary.

In [ ]:
### BEGIN Solution
x_context = None

x_response = None

# Join the feature matrix
x_train = None

### END Solution

Generate the response and contet test sets with their **relevant** feature encodings.

In [ ]:
### BEGIN Solution
x_context_test = None

x_response_test = None

x_test = None

### END Solution

Select the best linear model among $C \in [\mathtt{1.e-3}, .., \mathtt{1.e3}]$.

In [ ]:
### BEGIN Solution
# pick the best C

best_linear_model = None

### END Solution

Select the best RBF kernel model among the hyperparameters
$C \in [\mathtt{1.e-3}, .., \mathtt{1.e3}]$ and $\gamma \in [\mathtt{1.e-3}, .., \mathtt{1.e3}]$.

In [ ]:
### BEGIN Solution
# pick the best C and gamma

best_kernel_model = None

### END Solution

Plot the `ROC` curves of both models and their `ROC AUC` scores.

In [ ]:
### BEGIN Solution

### END Solution

... and the best model is...?

In [ ]:
best_model = None
test_predictions = best_model.decision_function(x_test)
test_score = roc_auc_score(y_test, test_predictions)
print("Test ROC AUC = {}".format(test_score))

<br/> <!--Intentionally left blank-->